# Labeling Faces with Facial Recongition

This notebook utilizes the [facial recognition](https://github.com/ageitgey/face_recognition) library to recognize faces, encode the faces, and compare the encoded faces to known faces to find the closest match. The library itself handles the process of detecting faces and encoding them. This notebook builds a layer upon those two features to organize known users, attach multiple faces to a user, and label detected faces if we know the user.

The technology created in this notebook is a great starting place to build real world facial recognition technology, such as a door lock that recognizes a database of users.

Possible project ideas:
* Door lock that queries a database of users to find matches and unlocks the door if all conditions are met (Example conditions: No condition, time frame condition for workplace or rentals)

* Body cameras that queries a database of wanted individuals and notifies you if a wanted individual is found near you (Using wanted photos as user reference data) (Do you really want the police to use facial recognition though? Legality/ethics)

In [ ]:
!pip install https://github.com/ageitgey/face_recognition/archive/v1.2.2.tar.gz
!pip install opencv-python
!pip install 

     / 19.9MB 7.2MB/s
     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition: filename=face_recognition-1.2.2-py2.py3-none-any.whl size=15246 sha256=5d7c2a0fb1ede69f3252d402986bf4533a5989ab6d31d4f8d865d530c735e9db
  Stored in directory: /tmp/pip-ephem-wheel-cache-qszjljul/wheels/b1/ad/50/70c4119897208fd1bd524711e9fff3400b1621a769a42fe34f
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=d55e9a65ac7ae8b740f959e478901bf18d4f3b53053aaa3c348bf2326086177a
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition face-recognition-models


### Runtime Layer

This is the layer that allows a runtime to create an array of **User** objects (all of your users) and compare them all to a possible face. If none of the users are matches, the method returns None. Otherwise, the user with the lowest distance to the possible face is the match.

In [ ]:
import numpy as np
import face_recognition

In [ ]:
# A user will have an ID, a name (for labeling), and an array of face vectors
class User:
  def __init__(self, uuid, name, vectors=None):
    self.uuid = uuid
    self.name = name
    self.vectors = vectors
    if self.vectors is None or not isinstance(self.vectors, list): self.__init_vectors__()
  
  def __init_vectors__(self):
    self.vectors = []
  
  def add_vector(self, v):
    self.vectors.append(v)
    return self
  
  def remove_vector(self, v):
    self.vectors.remove(v)
    return self
  
  def match(self, v):
    return face_recognition.compare_faces(self.vectors, v)

  def distance(self, v):
    matches = self.match(v)
    if len(matches) <= 0: return None
    return np.min(face_recognition.face_distance(matches, v))

  # Gets the name of the closest user, or returns None if no match is found
  @staticmethod
  def calculate_label(users, v):
    # Calculate distance for all users
    matches = []
    for u in users:
      u_dist = u.distance(v) # Only include users we have a match for
      if u_dist is not None: matches.append((u, u_dist))
    
    # If we have no matches, we have no label
    if len(matches) <= 0: return None

    # Return the name of the closest face
    closest = min(matches, key=lambda x: x[1])
    return closest

### Example Implementation: Realtime Facial Detection & Labeling

We can use OpenCV to run our webcam and periodically grab frames, scan the frames for faces, and label the faces if we find a match.

In [39]:
import cv2

In [42]:
# Method to continuously scan my webcam and detect users as they are found
def webcam_detect(users=None, skip=2):
  # Create webcam feed
  video_capture = cv2.VideoCapture(0)

  # Continuously loop over the webcam feed and try and label faces
  fs = 0 # Frames skipped counter
  while True:
    # Get the RGB frame
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:,:,::-1]

    # If we haven't reached skip count yet, we should skip this frame
    if fs < skip:
        fs += 1
        continue
    # We've skipped enough frames. Process this frame and reset counter
    else: fs = 0

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    # Get names for faces
    names = []
    for v in face_encodings:
        u = User.calculate_label(users, v)
        if u is None: names.append('NOT A USER')
        else: names.append(u[0].name)

    # Label the detected faces with their name
    for (top, right, bottom, left), name in zip(face_locations, names):
      # Scale the images back to full
      top *= 4
      right *= 4
      bottom *= 4
      left *= 4

      cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255), 2)
      cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0,0,255), cv2.FILLED)
      cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255,255,255), 1)
    
    # Show the resulting image
    cv2.imshow('Video', frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'): break

  # Destroy the CV2 instances
  video_capture.release()
  cv2.destroyAllWindows()

In [ ]:
# A simple helper method to load and encode the ith face in a file (Typically files only have 1 face)
def encode_image(file, i=0):
  return face_recognition.face_encodings(face_recognition.load_image_file(file))[i]

In [ ]:
# Create an array of users where each user has an ID, name, and a number of vectors to which we can recognize them
users = [
         User(1, 'Ethan').add_vector(encode_image('ethan.png')),
         User(2, 'Ralph').add_vector(encode_image('ralph.png')),
         User(3, 'Joe Biden').add_vector(encode_image('joe.png'))
]

In [ ]:
# Run the detect webcam (You need an OpenCV capable machine to run this. Google Colab will not work!)
webcam_detect(users=users)

Results:

![](https://i.imgur.com/oPqSE9e.png)

### Final Thoughts

The majority of the stress comes from the iteration through all users. For a system to utilize facial recognition reliably, it must only process a small fraction of the frames captured by the sensor. This makes this library excellent for processing a set of pictures while being underwhelming for a video feed.

Ultimately, the future of facial recognition relies on the ability to attach a face to existing user data. Services such as Facebook and Instagram are in excellent positions to make this connection due to the already massive amount of pre-labeled data. This notebook is a great starting point for a bigger project.